In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import set_seed
import torch
from torch.optim import AdamW
from huggingface_hub import HfApi, login, hf_hub_download
from google.colab import userdata

from supplementary_file_for_sentence_similarity import *

# Loading the tokenizer and a dataset

In [2]:
# Loading the tokenizer

model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
dataloader = sentence_similarity_dataloaders(tokenizer)

set_seed(42)
train_dataloader, val_dataloader, test_dataloader = dataloader.get_dataloaders()

README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

# Training Experiments

In [ ]:
lr_list = [1e-5, 3e-5, 5e-5]
num_epochs = 3
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### With Random seed: 137

In [ ]:
for lr in lr_list:

  for lr_scheduler in [False, True]:

    set_seed(137)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    trainer_config = {'optimizer' : AdamW,
                  'num_epochs' : num_epochs,
                  'learning_rate' : lr,
                  'lr_scheduler' : lr_scheduler,
                  }

    trainer = sentence_similarity_trainer(model=model,
                      train_dataloader = train_dataloader,
                      val_dataloader = val_dataloader,
                      device = device,
                      trainer_config = trainer_config,
                      )

    # Running the training loops
    print("="*20, f"{lr=} and {lr_scheduler = }", "="*20)
    trainer.train()

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

==================== lr=1e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.6667869357242044 --validation loss: 0.6652079060965893 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.6108060662263359 --validation loss: 0.604514034355388 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.5875314903285249 --validation loss: 0.5804770939490375 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.5104838929794453 --validation loss: 0.5008577152210123 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.47521460546517424 --validation loss: 0.47357424684599336 -- validation accuracy 0.7573529411764706
Epoch 0 Step 250 -- training loss: 0.3851829305583355 --validation loss: 0.39071178786894856 -- validation accuracy 0.8382352941176471
Epoch 0 Step 300 -- training loss: 0.3571056387425768 --validation loss: 0.39046551915360433 -- validation accuracy 0.838

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

==================== lr=3e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6591756247746918 --validation loss: 0.6570563000791213 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.5996672137110841 --validation loss: 0.5985263989252203 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6024950027206105 --validation loss: 0.5991765026952706 -- validation accuracy 0.6862745098039216
Epoch 0 Step 150 -- training loss: 0.5394540047658555 --validation loss: 0.5386286924282709 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.49501757180898537 --validation loss: 0.4905316233634949 -- validation accuracy 0.7450980392156863
Epoch 0 Step 250 -- training loss: 0.49872024149546695 --validation loss: 0.5059807034684163 -- validation accuracy 0.7450980392156863
Epoch 0 Step 300 -- training loss: 0.3676024107404524 --validation loss: 0.38787424403662774 -- validation accuracy 0.8137

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=5e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.6530743077689526 --validation loss: 0.6504536878828909 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.6041894336931066 --validation loss: 0.6024449187166551 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.5668509719418544 --validation loss: 0.5647742415175718 -- validation accuracy 0.7205882352941176
Epoch 0 Step 150 -- training loss: 0.42323150146501 --validation loss: 0.42123525399787753 -- validation accuracy 0.821078431372549
Epoch 0 Step 200 -- training loss: 0.4348972865617743 --validation loss: 0.4295421901578997 -- validation accuracy 0.8088235294117647
Epoch 0 Step 250 -- training loss: 0.40494480410951933 --validation loss: 0.411915618707152 -- validation accuracy 0.8161764705882353
Epoch 0 Step 300 -- training loss: 0.3825738120718589 --validation loss: 0.39533209844547157 -- validation accuracy 0.8210784

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=5e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6530743077689526 --validation loss: 0.6504536878828909 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.5885026410773948 --validation loss: 0.5932766123145234 -- validation accuracy 0.6936274509803921
Epoch 0 Step 100 -- training loss: 0.4981065565204828 --validation loss: 0.5012698307925579 -- validation accuracy 0.7671568627450981
Epoch 0 Step 150 -- training loss: 0.6168347671465676 --validation loss: 0.6194208413362503 -- validation accuracy 0.6936274509803921
Epoch 0 Step 200 -- training loss: 0.4332171979163467 --validation loss: 0.42427294160805495 -- validation accuracy 0.7965686274509803
Epoch 0 Step 250 -- training loss: 0.3859522763582876 --validation loss: 0.3772841899418363 -- validation accuracy 0.8357843137254902
Epoch 0 Step 300 -- training loss: 0.42545673160251707 --validation loss: 0.4165697752260694 -- validation accuracy 0.86274

**Observations**:

In the first experiment, we see overfitting after epoch 1. The best result for this experiment is after step 400 of epoch 1, where the validation accuracy was 88.2 and the validation loss is 0.264.

In the second experiment, we again see overfitting after epoch 1. The best result for for this experiment is at step 458 of epoch 1, where the validation accuracy is 88.5 and the validation loss is 0.277.

In the third experiment, both the train and validation losses starts osciallting after step 300 of epoch 1. Until then, both losses follow the same pattern. Step 300 of epoch 1 is also the step where we have achieve the best result: validation accuracy of 90.0 and the validation loss is 0.276. Thus, we deduce that training until step 300 of epoch 1 is the best option.

In the fourth experiment, we can see clear signs of overfitting after epoch 1. Even during epoch 1, we see some oscillations in both train and validation losses, but the losses go down overall. The best result that we get in this experiment is after step 450 of epoch 1, where the validation accuracy is 89.5 and the validation loss is 0.287.

In the fifth experiment, we see osciallions in the train and validation losses even in the first epoch. This suggests that the constant learning rate of $5\times 10^{-5}$ is too large, and hence, we do not consider this experiment.

In the sixth experiment, we also see some osillations in the first few steps of epoch 0, though the general trend is losses going down. The best result is after step 458 of epoch 0 as we see some overfitting after step 100 of epoch 1. The validation accuracy and the validation loss after step 458 of epoch 0 are 88.7 and 0.274, respectively.


Based on these observations, we conclude that the best result (validation accuracy of 90.0 and validation loss 0.276) is achieved with experiment 3 if we stop after step 300 of epoch 1.

### With Random seed: 23

In [ ]:
for lr in lr_list:

  for lr_scheduler in [False, True]:

    set_seed(23)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    trainer_config = {'optimizer' : AdamW,
                  'num_epochs' : num_epochs,
                  'learning_rate' : lr,
                  'lr_scheduler' : lr_scheduler,
                  }

    trainer = sentence_similarity_trainer(model=model,
                      train_dataloader = train_dataloader,
                      val_dataloader = val_dataloader,
                      device = device,
                      trainer_config = trainer_config,
                      )

    # Running the training loops
    print("="*20, f"{lr=} and {lr_scheduler = }", "="*20)
    trainer.train()

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=1e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.7142540099574071 --validation loss: 0.7153000247244742 -- validation accuracy 0.3161764705882353
Epoch 0 Step 50 -- training loss: 0.570311360726689 --validation loss: 0.560642758713049 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.5881208696915953 --validation loss: 0.5761717227744121 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.5373942655285978 --validation loss: 0.5286628329286388 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.5190942451065662 --validation loss: 0.5136880085748785 -- validation accuracy 0.696078431372549
Epoch 0 Step 250 -- training loss: 0.4686998418311148 --validation loss: 0.46450843296799005 -- validation accuracy 0.7769607843137255
Epoch 0 Step 300 -- training loss: 0.4501490399144054 --validation loss: 0.4550619046477711 -- validation accuracy 0.75
Epoch

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

==================== lr=3e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.7021817983105811 --validation loss: 0.7027251720428467 -- validation accuracy 0.3161764705882353
Epoch 0 Step 50 -- training loss: 0.627975876359898 --validation loss: 0.6220578364297455 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6149656856371686 --validation loss: 0.6084020202066384 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.6438113289980587 --validation loss: 0.6429845246614194 -- validation accuracy 0.6911764705882353
Epoch 0 Step 200 -- training loss: 0.46577885591126733 --validation loss: 0.4613719328361399 -- validation accuracy 0.7107843137254902
Epoch 0 Step 250 -- training loss: 0.48408761980876425 --validation loss: 0.4862403764444239 -- validation accuracy 0.7818627450980392
Epoch 0 Step 300 -- training loss: 0.3283048054783692 --validation loss: 0.35080594248047064 -- validation accuracy 0.85049

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=5e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.69170141479808 --validation loss: 0.6917593350597456 -- validation accuracy 0.7009803921568627
Epoch 0 Step 50 -- training loss: 0.6289774116477675 --validation loss: 0.6248316677177653 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6403157966848552 --validation loss: 0.6374106991524789 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.6214604548509345 --validation loss: 0.6192934150789299 -- validation accuracy 0.6838235294117647
Epoch 0 Step 200 -- training loss: 0.6368788792416941 --validation loss: 0.6465162424480214 -- validation accuracy 0.6936274509803921
Epoch 0 Step 250 -- training loss: 0.6046194216654451 --validation loss: 0.6004008937115762 -- validation accuracy 0.6838235294117647
Epoch 0 Step 300 -- training loss: 0.6195854262199277 --validation loss: 0.6125239244863099 -- validation accuracy 0.68382352

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=5e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.69170141479808 --validation loss: 0.6917593350597456 -- validation accuracy 0.7009803921568627
Epoch 0 Step 50 -- training loss: 0.620305874248995 --validation loss: 0.6138201531241921 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6259580749312258 --validation loss: 0.6200506804036159 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.5078704053605044 --validation loss: 0.48243172934242323 -- validation accuracy 0.7892156862745098
Epoch 0 Step 200 -- training loss: 0.4566218298504815 --validation loss: 0.46827417889646455 -- validation accuracy 0.6838235294117647
Epoch 0 Step 250 -- training loss: 0.4288787281136627 --validation loss: 0.41813012665393307 -- validation accuracy 0.8357843137254902
Epoch 0 Step 300 -- training loss: 0.354830644505346 --validation loss: 0.3623017443745744 -- validation accuracy 0.86029411

**Observations**:

In experiment 1, we see a monotonic decrease in the train and validation losses until step 150 of epoch 1. After epoch 1, we see clear signs of overfitting. The best result is achieved at step 150 of epoch 1, where the validation accuracy is 89.7 and the validation loss is 0.280.

In experiment 2, we see overfitting after epoch 1. Even though the decrease in losses in not monotonic in epoch 1, both losses follow the same pattern. We conclude that the best result in this experiment is achieved at step 458 of epoch 1. The validation accuracy at this step is 89.7 and the validation loss is 0.269.

In experiment 3, we see overfitting even after epoch 0, which suggests that the constant learning rate of $3\times 10^{-5}$ is too large. The best result is at the end (i.e. step 458) of epoch 0 where the validation accuracy is 88.0 and the validation loss is 0.270.

In experiment 4, the best result is at step 100 of epoch 1. After this step, we see signs of overfitting. The validation accuracy and the validation loss at this step are 88.0 and 0.267, respectively.

In experiment 5, we do not see any significant learning. The train and validation loss has some oscillations, but losses remain constant overall. The validation accuracy also remained a constant.

In experiment 6, we see some oscillations in the lossed in epoch 1, but the overall losses decrease in epoch 1. There is overfitting in epoch 2. The best result is validation accuracy of 90.4 and validation loss of 0.228 at step 458 of epoch 1.

Therefore, we conclude that the best result (validation accuracy of 90.4 and validation loss of 0.228) is achieved with experiment 6 when we stop after step 458 of epoch 1.

### With Random seed: 114

In [ ]:
for lr in lr_list:

  for lr_scheduler in [False, True]:

    set_seed(114)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    trainer_config = {'optimizer' : AdamW,
                  'num_epochs' : num_epochs,
                  'learning_rate' : lr,
                  'lr_scheduler' : lr_scheduler,
                  }

    trainer = sentence_similarity_trainer(model=model,
                      train_dataloader = train_dataloader,
                      val_dataloader = val_dataloader,
                      device = device,
                      trainer_config = trainer_config,
                      )

    # Running the training loops
    print("="*20, f"{lr=} and {lr_scheduler = }", "="*20)
    trainer.train()

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=1e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.657562812818681 --validation loss: 0.6554916524419597 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.5899746712776051 --validation loss: 0.5860051968518425 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.5644156907508576 --validation loss: 0.5607577965540045 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.5146728846891773 --validation loss: 0.5062470073793449 -- validation accuracy 0.8014705882352942
Epoch 0 Step 200 -- training loss: 0.46132878788740805 --validation loss: 0.4713859317057273 -- validation accuracy 0.7720588235294118
Epoch 0 Step 250 -- training loss: 0.42014091817382115 --validation loss: 0.4205445559585796 -- validation accuracy 0.803921568627451
Epoch 0 Step 300 -- training loss: 0.3693968091233104 --validation loss: 0.38690899327105166 -- validation accuracy 0.82843

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

==================== lr=3e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6503948523847626 --validation loss: 0.6477908270031798 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.5635546353160685 --validation loss: 0.5734213275067946 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.5559465202760593 --validation loss: 0.5535504338788051 -- validation accuracy 0.7769607843137255
Epoch 0 Step 150 -- training loss: 0.40484985783046884 --validation loss: 0.410572336584914 -- validation accuracy 0.821078431372549
Epoch 0 Step 200 -- training loss: 0.3122581255364522 --validation loss: 0.3331249437349684 -- validation accuracy 0.8578431372549019
Epoch 0 Step 250 -- training loss: 0.2841679236094806 --validation loss: 0.35342315224674986 -- validation accuracy 0.8651960784313726
Epoch 0 Step 300 -- training loss: 0.24771744119965172 --validation loss: 0.289398949465477 -- validation accuracy 0.8897058

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=5e-05 and lr_scheduler = False ====================
Epoch 0 Step 0 -- training loss: 0.6437579537528793 --validation loss: 0.6405308632289662 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.5854075681539922 --validation loss: 0.5804453649941612 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6949990572233345 --validation loss: 0.697086355265449 -- validation accuracy 0.4362745098039216
Epoch 0 Step 150 -- training loss: 0.4270197641356059 --validation loss: 0.41953683542270287 -- validation accuracy 0.8357843137254902
Epoch 0 Step 200 -- training loss: 0.3696685207516669 --validation loss: 0.361798552905812 -- validation accuracy 0.8602941176470589
Epoch 0 Step 250 -- training loss: 0.29921283268152626 --validation loss: 0.3049929860029735 -- validation accuracy 0.8725490196078431
Epoch 0 Step 300 -- training loss: 0.3199972248715318 --validation loss: 0.36409587895168977 -- validation accuracy 0.86764

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==================== lr=5e-05 and lr_scheduler = True ====================
Epoch 0 Step 0 -- training loss: 0.6437579537528793 --validation loss: 0.6405308632289662 -- validation accuracy 0.6838235294117647
Epoch 0 Step 50 -- training loss: 0.5449232753845081 --validation loss: 0.5432528774527943 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.7164638395662661 --validation loss: 0.7205202334067401 -- validation accuracy 0.3946078431372549
Epoch 0 Step 150 -- training loss: 0.38416464573222825 --validation loss: 0.3925265924018972 -- validation accuracy 0.7965686274509803
Epoch 0 Step 200 -- training loss: 0.3352461500430159 --validation loss: 0.35935068641807516 -- validation accuracy 0.8553921568627451
Epoch 0 Step 250 -- training loss: 0.30024435040327946 --validation loss: 0.3468008282612644 -- validation accuracy 0.8700980392156863
Epoch 0 Step 300 -- training loss: 0.2817195933433919 --validation loss: 0.3196397645435497 -- validation accuracy 0.8651

**Observations**:

In experiment 1, we see some oscillations in the validation loss in epoch 1, but the validation loss goes down overall. After epoch 1, we see clear signs of overfitting. The best result in this experiment is after step 458 of epoch 1, where the validation accuracy is 88.7 and validation loss is 0.255.

In experiment 2, we see overfitting in epoch 2. Even though there are oscillations in both of the losses in epoch 1, the losses follow the same pattern and they both go down overall. The result is at step 458 of epoch 1, where the validation accuracy is 89.3 and the validation loss is 0.270.

In experiment 3, we see the best result at step 200 of epoch 1, after which we see signs of overfitting. The validation accuracy and validation loss at this step are 89.7 and 0.267, respectively.

In experiment 4, we see overfitting after epoch 0, which is slightly confusing. The best result is at step 458 of epoch 0, where the validation accuracy is 89.0 and the validation loss is 0.253.

In experiment 5, we see oscillations in the train and validation losses even in epoch 0. Nevertheless, the losses overall go down in epoch 0 and 1. The best result is at step 450 of epoch 1, where the validation accuracy is 89.0 and the validation loss is 0.253.

In experiment 6, we see overfitting after step 250 of epoch 1. Until this step, both losses follow the same pattern and they overall go down. The best result is also at this step, where the validation accuracy and validation loss are 90.7 and 0.253, respectively.

Based on this analysis, we conclude that the best result are acheived when we stop experiment 6 after step 250 of epoch 1.

# Conclusion

We conclude that the experiment with a random seed of 23 with a linearly decreasing learning rate from $5\times 10^{-5}$ to zero led to the best result if we stop after step 458 of epoch 1.

In [5]:
'''
Repeating the best performing experiment
with the stopping condition to get the
model weights and to calculate test accuracy.
'''

lr = 5e-5
lr_scheduler = True
num_epochs = 3
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
stopping_condition = {'step': 458, 'epoch': 1}

set_seed(23)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

trainer_config = {'optimizer' : AdamW,
              'num_epochs' : num_epochs,
              'learning_rate' : lr,
              'lr_scheduler' : lr_scheduler,
              }



trainer = sentence_similarity_trainer(model=model,
                  train_dataloader = train_dataloader,
                  val_dataloader = val_dataloader,
                  device = device,
                  trainer_config = trainer_config,
                  stopping_condition = stopping_condition,
                  )

# Running the training loops
trainer.train()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 0 Step 0 -- training loss: 0.69170141479808 --validation loss: 0.6917593350597456 -- validation accuracy 0.7009803921568627
Epoch 0 Step 50 -- training loss: 0.620305874248995 --validation loss: 0.6138201531241921 -- validation accuracy 0.6838235294117647
Epoch 0 Step 100 -- training loss: 0.6259580749312258 --validation loss: 0.6200506804036159 -- validation accuracy 0.6838235294117647
Epoch 0 Step 150 -- training loss: 0.5078704053605044 --validation loss: 0.48243172934242323 -- validation accuracy 0.7892156862745098
Epoch 0 Step 200 -- training loss: 0.4566218298504815 --validation loss: 0.46827417889646455 -- validation accuracy 0.6838235294117647
Epoch 0 Step 250 -- training loss: 0.4288787281136627 --validation loss: 0.41813012665393307 -- validation accuracy 0.8357843137254902
Epoch 0 Step 300 -- training loss: 0.354830644505346 --validation loss: 0.3623017443745744 -- validation accuracy 0.8602941176470589
Epoch 0 Step 350 -- training loss: 0.36040909267345156 --validatio

In [6]:
'''
Calculating the test loss
'''

def test_evaluation():

    trainer.model.eval()
    with torch.no_grad():

      test_losses = []
      test_accuracies = []

      for i, batch in enumerate(test_dataloader):

        # Getting the batch loss
        batch = {k: v.to(trainer.device) for k, v in batch.items()}
        outputs = trainer.model(**batch)
        test_losses.append(outputs.loss.item())
        # Getting the batch accuracy
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        test_accuracy = (predictions == batch['labels']).float().mean()
        test_accuracies.append(test_accuracy.item())

      avg_test_loss = sum(test_losses) / len(test_losses)
      avg_test_accuracy = sum(test_accuracies) / len(test_accuracies)

    trainer.model.train()

    return avg_test_loss, avg_test_accuracy

test_loss, test_acc = test_evaluation()
print(f"{test_loss=}")
print(f"{test_acc=}")

test_loss=0.32305042116247396
test_acc=0.8715277777777778


In [7]:
'''
Uploading the weights on HF
'''

# Save model weights
file_name = "model_weights.pth"
trainer.save_model(file_name)

# Logging into Hugging face Hub
hf_token = userdata.get('hf_TOKEN')
login(token=hf_token)
api = HfApi()

repo_id = "mudassirmoosa/sentence-similarity-transformer-comparison"

# Uploading model weights
api.upload_file(
    path_or_fileobj=file_name,
    path_in_repo="DeBERTa_for_sentence_similarity.pth",
    repo_id=repo_id,
    token=hf_token
)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  model_weights.pth                     :   0%|          |  555kB /  738MB            

CommitInfo(commit_url='https://huggingface.co/mudassirmoosa/sentence-similarity-transformer-comparison/commit/2b526901bcf6255d37b998a3adcf00a9b006ff02', commit_message='Upload DeBERTa_for_sentence_similarity.pth with huggingface_hub', commit_description='', oid='2b526901bcf6255d37b998a3adcf00a9b006ff02', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mudassirmoosa/sentence-similarity-transformer-comparison', endpoint='https://huggingface.co', repo_type='model', repo_id='mudassirmoosa/sentence-similarity-transformer-comparison'), pr_revision=None, pr_num=None)